## Packages

In [ ]:
import pandas as pd
import numpy as np
from math import ceil
import sqlalchemy 
from sqlalchemy import create_engine, text
from db_secrets import SQL_107

## Test Connection

In [ ]:
## text for query
query_text = """
             SELECT * 
                 , CONVERT(DATETIME,AE_Arrival,21) AS [AE_Arrival_Datetime]
                 , CAST(DATEDIFF(MINUTE
                             ,[Call Connect Time]
                             ,CONVERT(DATETIME,AE_Arrival,21) )
                        AS float)/60.0 AS [Hours_to_AE]
             FROM [ReportingGateway].[dbo].[uec_111_V2]
             WHERE [CallDate] >= '2023-01-01'
             """

In [ ]:
## Function to round up to the nearest 5 for small number supression
def round_up_to_5(x):
        return ceil(x / 5) * 5

In [ ]:
## Create an engine + connection
engine = create_engine(SQL_107())
conn = engine.connect()

## Return data
df_raw = pd.read_sql(query_text,conn)

In [ ]:
## Makes working copy
df = df_raw.copy()

In [ ]:
def Trauma_Detect(x):
    if 'Blunt' in x:
        return 'Blunt'
    elif 'Penetrating' in x:
        return 'Penetrating' 
    elif  'Trauma' in x:
        return 'Other Trauma'
    else:
        return 'Not Trauma'

In [ ]:
def Pregnancy_Detect(x):
    if 'Pregnant, Over 20 Weeks' in x:
        return 'Over 20 Weeks'
    elif 'Pregnant, Under 20 Weeks' in x:
        return 'Under 20 Weeks' 
    elif  'Pregnant' in x:
        return 'Other Pregnancy'
    else:
        return 'Not Pregnant'

In [ ]:
df.columns

In [ ]:
df.loc[:,'Call_Taker_Triage'] = df['Call_Taker_Triages'].transform(lambda x: 'No' if x == 0 else 'Yes')
df.loc[:,'Clinical_Triage'] = df['Clinical_Triages'].transform(lambda x: 'No' if x == 0 else 'Yes')

df.loc[:,"Trauma_Type"] = df["SymptomGroup"].transform(lambda x: Trauma_Detect(x))
df.loc[:,"Pregnant"] = df["SymptomGroup"].transform(lambda x: Pregnancy_Detect(x))

df.loc[:,'SymptomGroup'] = df['SymptomGroup'].str.replace(', Blunt', '', regex=True)
df.loc[:,'SymptomGroup'] = df['SymptomGroup'].str.replace(', Penetrating', '', regex=True)
df.loc[:,'SymptomGroup'] = df['SymptomGroup'].str.replace(', Pregnant, Over 20 Weeks', '', regex=True)
df.loc[:,'SymptomGroup'] = df['SymptomGroup'].str.replace(', Pregnant, Under 20 Weeks', '', regex=True)
df.loc[:,'SymptomGroup'] = df['SymptomGroup'].str.replace(', Pregnant', '', regex=True)

df.loc[:,'AE_Outcome'] = (df.loc[:,'AE_Outcome']
                          .transform(lambda x: x or None) ## replaces empty strings with None
                          .fillna('No UEC Contact')
                          )
df.loc[:,'Indicator Type'] = (df.loc[:,'Indicator Type']
                          .fillna('No UEC Contact'))


In [ ]:
def generate_table1(data,group = None):

    ## Number of rows
    N_row = len(data)
     
    ## Separate numerical and categorical columns
    numeric_cols = data.select_dtypes(include=['number']).columns
    categorical_cols = data.select_dtypes(include=['object', 'category', 'bool']).columns
    
     ## filters to remove group, creates group item values
    if group:
        categorical_cols = [x for x in categorical_cols if x != group]
        group_counts = df[group].value_counts()
        items = (data.groupby(group)
                    .size()
                    .to_frame(name='count')
                    .sort_values('count', ascending=False)
                    .index)
    
    ## Initialize an empty list to collect rows for the summary DataFrame
    summary_rows = []

    ## Function to format median and quartiles
    def median_iqr(col):
        return f"{col.median():.1f} ({col.quantile(0.25):.1f},{col.quantile(0.75):.1f})"
    
    ## Function to format number and percent
    def count_pc(data,col,level,group = None,group_counts = None,item = None):
        if group:
            count = data.loc[data[group] == item, col].value_counts().get(level,0)
            percent = (count / group_counts.get(item,0)) * 100
        else:
            count = data[col].value_counts().get(level, 0)
            percent = (count / N_row) * 100
        return f'{round_up_to_5(count):,} ({percent:.1f}%)'          

    # Numeric data summary (median IQR)
    for col in numeric_cols:
        row = {'Characteristic': f'{col} (median, IQR)'
               ,'Level': ''}
        if group:  
            for item in items:               
                item_value = median_iqr(data.loc[data[group] == item, col])
                row[f'{item}, N={round_up_to_5(group_counts.get(item,0)):,}'] = item_value
        row[f'Overall, N={round_up_to_5(N_row):,}'] = median_iqr(data[col])
        summary_rows.append(row)

    # Categorical data summary (each category level on a separate line)
    for col in categorical_cols:       
        levels = (data.groupby(col)
                    .size()
                    .to_frame(name='count')
                    .sort_values('count', ascending=False)
                    .index)

        for level in levels:
            row = { 'Characteristic': f'{col} (N, %)'
                , 'Level': f'{level}'}
            if group:  
                for item in items:
                    item_value = count_pc(data,col,level,group,group_counts,item)
                    row[f'{item}, N={round_up_to_5(group_counts.get(item,0)):,}'] = item_value
            row[f'Overall, N={round_up_to_5(N_row):,}'] = count_pc(data,col,level)
            summary_rows.append(row)


    # Convert the summary rows to a DataFrame
    table1_df = pd.DataFrame(summary_rows).set_index('Characteristic')
    return table1_df



In [ ]:
df_table_one = df[['Disposition_Group',
       'In_Out_Hours', 'Call_Taker_Triage',
       'Clinical_Triage', 'Patient Age',
       'AE_Outcome', 'Indicator Type','Hours_to_AE','UEC_Lookup']].copy()

table_one = generate_table1(df_table_one,group='AE_Outcome')

table_one

In [ ]:
df_table_two = df[['Trauma_Type','Pregnant','SymptomGroup','AE_Outcome']].copy()

table_two = generate_table1(df_table_two,group='AE_Outcome')

table_two